<a href="https://colab.research.google.com/github/jlonge4/gen_ai_utils/blob/main/phi3_5_rag_relevance_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets accelerate peft bitsandbytes flash-attn pycm transformers

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline
import torch

base_model_id = "microsoft/Phi-3.5-mini-instruct"
groundedai_eval_id = "grounded-ai/phi3.5-rag-relevance-judge"

if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
# If bfloat16 is not supported, 'compute_dtype' is set to 'torch.float16' and 'attn_implementation' is set to 'sdpa'.
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

config = PeftConfig.from_pretrained(groundedai_eval_id)
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, attn_implementation=attn_implementation,torch_dtype=compute_dtype)
model_peft = PeftModel.from_pretrained(base_model, groundedai_eval_id, config=config)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

merged_model = model_peft.merge_and_unload()
merged_model.to('cuda')

In [3]:
def format_func(query, text):
    input = f"""
          You are comparing a reference text to a question and trying to determine if the reference text
      contains information relevant to answering the question. Here is the data:
          [BEGIN DATA]
          ************
          [Question]: {query}
          ************
          [Reference text]: {text}
          ************
          [END DATA]
      Compare the Question above to the Reference text. You must determine whether the Reference text
      contains information that can answer the Question. Please focus on whether the very specific
      question can be answered by the information in the Reference text.
      Your response must be single word, either "relevant" or "unrelated",
      and should not contain any text or characters aside from that word.
      "unrelated" means that the reference text does not contain an answer to the Question.
      "relevant" means the reference text contains an answer to the Question.
      Based on the information provided, is the provided reference relevant to the question? Respond with only "relevant" or "unrelated"
  """
    return input

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
tokenizer.padding_side = 'right'

def run_merged_model(query, text):
  input = format_func(query, text)
  messages = [
      {"role": "user", "content": input}
  ]

  pipe = pipeline(
      "text-generation",
      model=merged_model,
      tokenizer=tokenizer,
      device='cuda'
  )

  generation_args = {
      "max_new_tokens": 2,
      "return_full_text": False,
      "temperature": 0.01,
      "do_sample": True,
  }

  output = pipe(messages, **generation_args)
  torch.cuda.empty_cache()
  return output[0]['generated_text'].strip().lower()

In [10]:
q = "What are the health benefits of drinking green tea?"
# Retrieved Text 1 (Related):
rt_1 = '''
Green tea is rich in polyphenols, particularly catechins, which are powerful antioxidants. Regular consumption of green tea has been associated with various health benefits, including improved brain function,
 fat loss, protection against cancer, and a lower risk of heart disease. Some studies suggest that the compounds in green tea can boost metabolic rate and increase fat burning. Additionally, the L-theanine in
  green tea may have a calming effect on the mind.
'''
# Retrieved Text 2 (Unrelated):
rt_2 = '''The Great Barrier Reef is the world's largest coral reef system, stretching over 2,300 kilometers off the coast of Australia. It comprises over 2,900 individual reefs and 900 islands. The reef is home
 to an incredibly diverse ecosystem, including over 1,500 species of fish, 400 types of hard coral, and numerous other marine species. However, climate change and ocean acidification pose significant threats to the
  reef's health and survival.'''
# Retrieved Text 3 (Unrelated):
rt_3 = '''The Industrial Revolution, which began in Britain in the late 18th century, marked a major turning point in human history. It saw the transition from hand production methods to machines, new chemical manufacturing
 and iron production processes, improved efficiency of water power, the increasing use of steam power, and the development of machine tools. This period of rapid industrial growth transformed economic and social conditions,
  leading to urbanization and significant technological innovations.'''
# hard case (unrelated)
rt_4 = '''Green tea production is a meticulous process that begins with the careful selection of tea leaves. The leaves are typically harvested from Camellia sinensis plants, with the best quality teas using only the bud and
first two leaves. After harvesting, the leaves undergo minimal oxidation through a process of withering and steaming or pan-firing. This halts the oxidation process and preserves the tea's natural green color and delicate
 flavor profile. The leaves are then rolled and dried to create the final product. Different regions, particularly in China and Japan, have developed unique processing methods that result in various green tea varieties,
 each with its distinct characteristics and flavor notes.'''

In [12]:
#relevant
run_merged_model(q, rt_1)

'relevant'

In [7]:
#unrelated
run_merged_model(q, rt_2)

'unrelated'

In [8]:
run_merged_model(q, rt_3)

'unrelated'

In [11]:
# hard case
run_merged_model(q, rt_4)

'unrelated'